<a href="https://colab.research.google.com/github/Rakesh2711/Revoultbankexam/blob/master/Revoult_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os
import pandas_profiling as pp
import seaborn as sns
# from ggplot import *
import matplotlib.pyplot as plt
from collections import Counter
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
%matplotlib inline


In [2]:
os.getcwd()

'/content'

In [0]:
os.chdir('/content/drive/My Drive/data')

In [0]:
# # Read 4 files

device = pd.read_csv('rev-devices.csv')
notification =  pd.read_csv('rev-notifications.csv')
transaction = pd.read_csv('rev-transactions.csv')
users =  pd.read_csv('rev-users.csv')

In [5]:
device.head()

,brand,user_id
0,Android,user_3257
1,Apple,user_6809
2,Android,user_9144
3,Apple,user_3636
4,Android,user_5970


In [6]:
notification.head()

,reason,channel,status,user_id,created_date
0,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_7086,2018-12-02 17:58:33.320645
1,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_6598,2018-12-01 23:09:37.367127
2,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_4151,2018-12-04 02:57:56.425660
3,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_1408,2018-12-11 02:04:46.284683
4,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_6292,2018-12-14 17:09:58.900808


In [7]:
transaction.head()

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,ea_cardholderpresence,ea_merchant_mcc,ea_merchant_city,ea_merchant_country,direction,user_id,created_date
0,transaction_0,TRANSFER,AED,4.55,COMPLETED,NaN,NaN,NaN,NaN,OUTBOUND,user_898,2018-04-03 03:34:21.784487
1,transaction_1,CARD_PAYMENT,AED,15.50,COMPLETED,FALSE,4111.0,Dubai,ARE,OUTBOUND,user_1652,2019-03-19 06:15:59.537032
2,transaction_2,CARD_PAYMENT,AED,43.40,COMPLETED,FALSE,5814.0,Dubai,ARE,OUTBOUND,user_1652,2019-03-18 18:53:41.323032
3,transaction_3,TRANSFER,AED,10043.01,COMPLETED,NaN,NaN,NaN,NaN,OUTBOUND,user_1652,2019-03-22 14:20:01.513032
4,transaction_4,CARD_PAYMENT,AED,43.81,COMPLETED,FALSE,5651.0,Abu Dhabi,ARE,OUTBOUND,user_5509,2019-03-22 05:42:50.316652


In [8]:
users.head()

,user_id,birth_year,country,city,created_date,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,num_referrals,num_successful_referrals
0,user_0,1989,PL,Gdansk,2018-01-13 05:15:15.599466,1,STANDARD,1.0,1.0,3,0,0
1,user_1,1975,GB,London,2018-01-29 03:38:46.676876,0,STANDARD,NaN,NaN,21,0,0
2,user_2,1987,PL,Poznań,2018-01-18 19:17:31.229096,0,STANDARD,0.0,0.0,21,0,0
3,user_3,1994,FR,Paris,2018-01-15 18:47:56.723104,0,STANDARD,1.0,0.0,0,0,0
4,user_4,1985,GB,Beckenham,2018-01-11 00:36:46.673673,0,STANDARD,NaN,NaN,2,0,0


In [9]:
transaction.isna().sum()

transaction_id                 0
transactions_type              0
transactions_currency          0
amount_usd                     0
transactions_state             0
ea_cardholderpresence    1170594
ea_merchant_mcc          1158658
ea_merchant_city         1158794
ea_merchant_country      1158979
direction                      0
user_id                        0
created_date                   0
dtype: int64

In [0]:
# Drpoing the columns which are having more than 40% of the data missing

for col in transaction.columns:
    if transaction[col].isna().sum()>(len(transaction)/100)*40:
        del transaction[col]

In [11]:
users.isna().sum()

user_id                                        0
birth_year                                     0
country                                        0
city                                           0
created_date                                   0
user_settings_crypto_unlocked                  0
plan                                           0
attributes_notifications_marketing_push     6610
attributes_notifications_marketing_email    6610
num_contacts                                   0
num_referrals                                  0
num_successful_referrals                       0
dtype: int64

In [12]:
# Missing value imputation for users table
users['attributes_notifications_marketing_push'].value_counts(dropna=False)/len(users)*100

1.0    62.521873
NaN    34.019557
0.0     3.458569
Name: attributes_notifications_marketing_push, dtype: float64

In [13]:
users['attributes_notifications_marketing_email'].value_counts(dropna=False)/len(users)*100

1.0    59.114771
NaN    34.019557
0.0     6.865672
Name: attributes_notifications_marketing_email, dtype: float64

In [0]:
# Here I am imputing the missing data with other catagory as 2 beacause I am trating the missing 
#catagory as other catagory

users.fillna(2,inplace=True)

# Till here we have imputed all the missing values 

In [0]:
notification['created_date']=notification['created_date'].astype('datetime64[ns]')

In [0]:
transaction['created_date'] = transaction['created_date'].astype('datetime64[ns]')

In [0]:
transaction['t_month'] = transaction['created_date'].dt.month

transaction['t_year'] = transaction['created_date'].dt.year

transaction['t_day'] = transaction['created_date'].dt.day

transaction['t_week'] = transaction['created_date'].dt.week

In [18]:
transaction.head()

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,direction,user_id,created_date,t_month,t_year,t_day,t_week
0,transaction_0,TRANSFER,AED,4.55,COMPLETED,OUTBOUND,user_898,2018-04-03 03:34:21.784487,4,2018,3,14
1,transaction_1,CARD_PAYMENT,AED,15.50,COMPLETED,OUTBOUND,user_1652,2019-03-19 06:15:59.537032,3,2019,19,12
2,transaction_2,CARD_PAYMENT,AED,43.40,COMPLETED,OUTBOUND,user_1652,2019-03-18 18:53:41.323032,3,2019,18,12
3,transaction_3,TRANSFER,AED,10043.01,COMPLETED,OUTBOUND,user_1652,2019-03-22 14:20:01.513032,3,2019,22,12
4,transaction_4,CARD_PAYMENT,AED,43.81,COMPLETED,OUTBOUND,user_5509,2019-03-22 05:42:50.316652,3,2019,22,12


In [0]:
del transaction['created_date']

In [0]:
notification['n_month'] = notification['created_date'].dt.month

notification['n_year'] = notification['created_date'].dt.year

notification['n_day'] = notification['created_date'].dt.day

notification['n_week'] = notification['created_date'].dt.week

del notification['created_date']

In [21]:
notification.head()

,reason,channel,status,user_id,n_month,n_year,n_day,n_week
0,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_7086,12,2018,2,48
1,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_6598,12,2018,1,48
2,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_4151,12,2018,4,49
3,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_1408,12,2018,11,50
4,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_6292,12,2018,14,50


In [0]:
users['created_date']=users['created_date'].astype('datetime64[ns]')

In [0]:
users['u_month'] = users['created_date'].dt.month

users['u_year'] = users['created_date'].dt.year

users['u_day'] = users['created_date'].dt.day

users['u_week'] = users['created_date'].dt.week

del users['created_date']

## Data PrePeration

In [24]:
list(zip([users.shape,transaction.shape,notification.shape,device.shape]))

[((19430, 15),), ((2740075, 11),), ((121813, 8),), ((19430, 2),)]

In [25]:
transaction.head(3)

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,direction,user_id,t_month,t_year,t_day,t_week
0,transaction_0,TRANSFER,AED,4.55,COMPLETED,OUTBOUND,user_898,4,2018,3,14
1,transaction_1,CARD_PAYMENT,AED,15.50,COMPLETED,OUTBOUND,user_1652,3,2019,19,12
2,transaction_2,CARD_PAYMENT,AED,43.40,COMPLETED,OUTBOUND,user_1652,3,2019,18,12


In [26]:
users.head(3)

,user_id,birth_year,country,city,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,num_referrals,num_successful_referrals,u_month,u_year,u_day,u_week
0,user_0,1989,PL,Gdansk,1,STANDARD,1.0,1.0,3,0,0,1,2018,13,2
1,user_1,1975,GB,London,0,STANDARD,2.0,2.0,21,0,0,1,2018,29,5
2,user_2,1987,PL,Poznań,0,STANDARD,0.0,0.0,21,0,0,1,2018,18,3


In [0]:
# pp.ProfileReport(users)

In [0]:
# attributes_notifications_marketing_email is highly correlated with attributes_notifications_marketing_push(ρ = 0.93666)
# num_contacts is highly Skewed
# num_referrals has constant value 0 
# num_successful_referrals has constant value 0 
# u_week is highly correlated with u_month (ρ = 0.98337) 
# I will drop num_referrals,num_successful_referrals,u_week from our dataset

In [0]:
# plt.rcParams['figure.figsize'] = 17,8
# sns.distplot(users['num_contacts'])
# plt.show()

In [30]:
users['num_referrals'].value_counts()

0    19430
Name: num_referrals, dtype: int64

In [31]:
users['num_successful_referrals'].value_counts()

0    19430
Name: num_successful_referrals, dtype: int64

In [0]:
users.drop(['num_referrals','num_successful_referrals','u_week'],axis=1,inplace=True)

In [33]:
users.head()

,user_id,birth_year,country,city,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,u_month,u_year,u_day
0,user_0,1989,PL,Gdansk,1,STANDARD,1.0,1.0,3,1,2018,13
1,user_1,1975,GB,London,0,STANDARD,2.0,2.0,21,1,2018,29
2,user_2,1987,PL,Poznań,0,STANDARD,0.0,0.0,21,1,2018,18
3,user_3,1994,FR,Paris,0,STANDARD,1.0,0.0,0,1,2018,15
4,user_4,1985,GB,Beckenham,0,STANDARD,2.0,2.0,2,1,2018,11


In [34]:
device.head()

,brand,user_id
0,Android,user_3257
1,Apple,user_6809
2,Android,user_9144
3,Apple,user_3636
4,Android,user_5970


In [0]:
# sns.countplot(device['brand'])

In [36]:
notification.head()

,reason,channel,status,user_id,n_month,n_year,n_day,n_week
0,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_7086,12,2018,2,48
1,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_6598,12,2018,1,48
2,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_4151,12,2018,4,49
3,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_1408,12,2018,11,50
4,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_6292,12,2018,14,50


In [0]:
# pp.ProfileReport(notification)

In [0]:
# user_id has a high cardinality: 18953 distinct values Warning
# n_week is highly correlated with n_month (ρ = 0.98474) Rejected
# Dataset has 2466 duplicate rows Warning

In [0]:
notification.drop('n_week',axis = 1,inplace = True)

In [40]:
# Before dropping the duplicate rows
notification.shape

(121813, 7)

In [0]:
notification.drop_duplicates(inplace=True)

In [42]:
# After dropping the duplicate rows
notification.shape

(119347, 7)

In [0]:
# pp.ProfileReport(transaction)


In [0]:
# amount_usd has 58604 / 2.1% zeros Zeros
# amount_usd is highly skewed (γ1 = 756.51) Skewed
# user_id has a high cardinality: 18766 distinct values Warning
# t_week is highly correlated with t_month (ρ = 0.98237) Rejected

In [0]:
transaction.drop('t_week',axis=1,inplace=True)

In [45]:
transaction['amount_usd'].min()

0.0

In [0]:
# sns.distplot(np.cbrt(transaction['amount_usd']),bins=5)
# plt.xlim(0,np.cbrt(85106453190.03))
# plt.show()

 # Joining Data For Data Preperation

In [47]:
df1 = pd.merge(users,device,on=['user_id'],how='inner')

df1.head()

,user_id,birth_year,country,city,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,u_month,u_year,u_day,brand
0,user_0,1989,PL,Gdansk,1,STANDARD,1.0,1.0,3,1,2018,13,Apple
1,user_1,1975,GB,London,0,STANDARD,2.0,2.0,21,1,2018,29,Apple
2,user_2,1987,PL,Poznań,0,STANDARD,0.0,0.0,21,1,2018,18,Android
3,user_3,1994,FR,Paris,0,STANDARD,1.0,0.0,0,1,2018,15,Apple
4,user_4,1985,GB,Beckenham,0,STANDARD,2.0,2.0,2,1,2018,11,Apple


In [48]:
df1.isna().sum()

user_id                                     0
birth_year                                  0
country                                     0
city                                        0
user_settings_crypto_unlocked               0
plan                                        0
attributes_notifications_marketing_push     0
attributes_notifications_marketing_email    0
num_contacts                                0
u_month                                     0
u_year                                      0
u_day                                       0
brand                                       0
dtype: int64

In [49]:
df2 = pd.merge(transaction,df1,on=['user_id'],how='left')

df2.head()

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,direction,user_id,t_month,t_year,t_day,birth_year,country,city,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,u_month,u_year,u_day,brand
0,transaction_0,TRANSFER,AED,4.55,COMPLETED,OUTBOUND,user_898,4,2018,3,1993,GB,Belfast,0,STANDARD,1.0,1.0,1,1,2018,11,Android
1,transaction_1,CARD_PAYMENT,AED,15.50,COMPLETED,OUTBOUND,user_1652,3,2019,19,1971,ES,Espartinas,1,PREMIUM_OFFER,1.0,1.0,13,3,2018,1,Apple
2,transaction_2,CARD_PAYMENT,AED,43.40,COMPLETED,OUTBOUND,user_1652,3,2019,18,1971,ES,Espartinas,1,PREMIUM_OFFER,1.0,1.0,13,3,2018,1,Apple
3,transaction_3,TRANSFER,AED,10043.01,COMPLETED,OUTBOUND,user_1652,3,2019,22,1971,ES,Espartinas,1,PREMIUM_OFFER,1.0,1.0,13,3,2018,1,Apple
4,transaction_4,CARD_PAYMENT,AED,43.81,COMPLETED,OUTBOUND,user_5509,3,2019,22,1992,PL,Siedlce,0,STANDARD,1.0,1.0,0,5,2018,18,Android


In [50]:
df2.isna().sum()

transaction_id                              0
transactions_type                           0
transactions_currency                       0
amount_usd                                  0
transactions_state                          0
direction                                   0
user_id                                     0
t_month                                     0
t_year                                      0
t_day                                       0
birth_year                                  0
country                                     0
city                                        0
user_settings_crypto_unlocked               0
plan                                        0
attributes_notifications_marketing_push     0
attributes_notifications_marketing_email    0
num_contacts                                0
u_month                                     0
u_year                                      0
u_day                                       0
brand                             

In [51]:
# We can see here for one user we have lot of records for the notifications sent

notification[notification['user_id']=='user_7086']

,reason,channel,status,user_id,n_month,n_year,n_day
0,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_7086,12,2018,2
3121,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_7086,12,2018,27
7077,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_7086,12,2018,21
18761,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_7086,1,2019,23
20151,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_7086,1,2019,11
21265,FIFTH_PAYMENT_PROMO,EMAIL,FAILED,user_7086,1,2019,17
28851,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_7086,11,2018,17
33362,FIFTH_PAYMENT_PROMO,PUSH,FAILED,user_7086,1,2019,17
55479,FIFTH_PAYMENT_PROMO,PUSH,SENT,user_7086,1,2019,17
59546,FIFTH_PAYMENT_PROMO,EMAIL,SENT,user_7086,1,2019,17


In [52]:
df3 = pd.merge(df2,notification,on='user_id',how='inner')

df3.head()

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,direction,user_id,t_month,t_year,t_day,birth_year,country,city,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,u_month,u_year,u_day,brand,reason,channel,status,n_month,n_year,n_day
0,transaction_0,TRANSFER,AED,4.55,COMPLETED,OUTBOUND,user_898,4,2018,3,1993,GB,Belfast,0,STANDARD,1.0,1.0,1,1,2018,11,Android,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,11,2018,28
1,transaction_0,TRANSFER,AED,4.55,COMPLETED,OUTBOUND,user_898,4,2018,3,1993,GB,Belfast,0,STANDARD,1.0,1.0,1,1,2018,11,Android,METAL_GAME_START,EMAIL,FAILED,2,2019,24
2,transaction_378,TRANSFER,AED,14.47,COMPLETED,OUTBOUND,user_898,4,2018,8,1993,GB,Belfast,0,STANDARD,1.0,1.0,1,1,2018,11,Android,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,11,2018,28
3,transaction_378,TRANSFER,AED,14.47,COMPLETED,OUTBOUND,user_898,4,2018,8,1993,GB,Belfast,0,STANDARD,1.0,1.0,1,1,2018,11,Android,METAL_GAME_START,EMAIL,FAILED,2,2019,24
4,transaction_379,ATM,AED,41.33,COMPLETED,OUTBOUND,user_898,4,2018,7,1993,GB,Belfast,0,STANDARD,1.0,1.0,1,1,2018,11,Android,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,11,2018,28


In [53]:
df3.shape

(17984549, 28)

In [0]:
# df3.isna().sum()

In [0]:
# df3[df3['user_id']=='user_898']

In [0]:
#
df_main = df3

In [57]:
df_main.head()

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,direction,user_id,t_month,t_year,t_day,birth_year,country,city,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,u_month,u_year,u_day,brand,reason,channel,status,n_month,n_year,n_day
0,transaction_0,TRANSFER,AED,4.55,COMPLETED,OUTBOUND,user_898,4,2018,3,1993,GB,Belfast,0,STANDARD,1.0,1.0,1,1,2018,11,Android,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,11,2018,28
1,transaction_0,TRANSFER,AED,4.55,COMPLETED,OUTBOUND,user_898,4,2018,3,1993,GB,Belfast,0,STANDARD,1.0,1.0,1,1,2018,11,Android,METAL_GAME_START,EMAIL,FAILED,2,2019,24
2,transaction_378,TRANSFER,AED,14.47,COMPLETED,OUTBOUND,user_898,4,2018,8,1993,GB,Belfast,0,STANDARD,1.0,1.0,1,1,2018,11,Android,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,11,2018,28
3,transaction_378,TRANSFER,AED,14.47,COMPLETED,OUTBOUND,user_898,4,2018,8,1993,GB,Belfast,0,STANDARD,1.0,1.0,1,1,2018,11,Android,METAL_GAME_START,EMAIL,FAILED,2,2019,24
4,transaction_379,ATM,AED,41.33,COMPLETED,OUTBOUND,user_898,4,2018,7,1993,GB,Belfast,0,STANDARD,1.0,1.0,1,1,2018,11,Android,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,11,2018,28


In [58]:
df_main.describe()

,amount_usd,t_month,t_year,t_day,birth_year,user_settings_crypto_unlocked,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,u_month,u_year,u_day,n_month,n_year,n_day
count,1.798455e+07,1.798455e+07,1.798455e+07,1.798455e+07,1.798455e+07,1.798455e+07,1.798455e+07,1.798455e+07,1.798455e+07,1.798455e+07,1.798455e+07,1.798455e+07,1.798455e+07,1.798455e+07,1.798455e+07
mean,1.803333e+05,5.976039e+00,2.018471e+03,1.547571e+01,1.985635e+03,3.495371e-01,1.216640e+00,1.177101e+00,2.871786e+01,5.761378e+00,2.018001e+03,1.556557e+01,5.841665e+00,2.018461e+03,1.615781e+01
std,9.728314e+07,3.658456e+00,4.991442e-01,8.895053e+00,9.854630e+00,4.768238e-01,5.474480e-01,5.956569e-01,6.112604e+01,3.229601e+00,3.544632e-02,8.846522e+00,4.155020e+00,4.984896e-01,8.713968e+00
min,0.000000e+00,1.000000e+00,2.018000e+03,1.000000e+00,1.929000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2.018000e+03,1.000000e+00,1.000000e+00,2.018000e+03,1.000000e+00
25%,2.150000e+00,3.000000e+00,2.018000e+03,8.000000e+00,1.981000e+03,0.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00,3.000000e+00,2.018000e+03,8.000000e+00,2.000000e+00,2.018000e+03,9.000000e+00
50%,8.510000e+00,5.000000e+00,2.018000e+03,1.500000e+01,1.988000e+03,0.000000e+00,1.000000e+00,1.000000e+00,1.300000e+01,6.000000e+00,2.018000e+03,1.600000e+01,4.000000e+00,2.018000e+03,1.700000e+01
75%,2.997000e+01,9.000000e+00,2.019000e+03,2.300000e+01,1.993000e+03,1.000000e+00,2.000000e+00,2.000000e+00,3.400000e+01,8.000000e+00,2.018000e+03,2.300000e+01,1.000000e+01,2.019000e+03,2.400000e+01
max,8.510645e+10,1.200000e+01,2.019000e+03,3.100000e+01,2.001000e+03,1.000000e+00,2.000000e+00,2.000000e+00,2.918000e+03,1.200000e+01,2.019000e+03,3.100000e+01,1.200000e+01,2.019000e+03,3.100000e+01


In [0]:
# Changing the Datatypes of below variables to Integer as the responses are 0 or 1
df_main['attributes_notifications_marketing_push'] = df_main['attributes_notifications_marketing_push'].astype('int64')
df_main['attributes_notifications_marketing_email'] = df_main['attributes_notifications_marketing_email'].astype('int64')

In [60]:
df_main.dtypes

transaction_id                               object
transactions_type                            object
transactions_currency                        object
amount_usd                                  float64
transactions_state                           object
direction                                    object
user_id                                      object
t_month                                       int64
t_year                                        int64
t_day                                         int64
birth_year                                    int64
country                                      object
city                                         object
user_settings_crypto_unlocked                 int64
plan                                         object
attributes_notifications_marketing_push       int64
attributes_notifications_marketing_email      int64
num_contacts                                  int64
u_month                                       int64
u_year      

# Exploratory Data Anaysis

In [61]:
df_main['user_settings_crypto_unlocked'].value_counts()

0    11698282
1     6286267
Name: user_settings_crypto_unlocked, dtype: int64

In [0]:
# plt.rcParams['figure.figsize'] = 13,6
# sns.countplot(x="transactions_type", hue="direction",data=df_main)
# plt.show()

In [0]:
# sns.countplot(x="transactions_type", hue="transactions_state",data=df_main)
# plt.show()

In [64]:
#Year Wise how many transactions 

pd.crosstab(df_main["t_year"],df_main["transactions_type"])

transactions_type,ATM,CARD_PAYMENT,CARD_REFUND,CASHBACK,EXCHANGE,FEE,REFUND,TAX,TOPUP,TRANSFER
t_year,,,,,,,,,,
2018,371755,5167305,38622,125405,629607,119025,7527,8710,1525198,1525022
2019,254096,4436018,38546,382575,443812,50707,2896,8556,1010927,1838240


In [65]:
df_main.head()

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,direction,user_id,t_month,t_year,t_day,birth_year,country,city,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,u_month,u_year,u_day,brand,reason,channel,status,n_month,n_year,n_day
0,transaction_0,TRANSFER,AED,4.55,COMPLETED,OUTBOUND,user_898,4,2018,3,1993,GB,Belfast,0,STANDARD,1,1,1,1,2018,11,Android,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,11,2018,28
1,transaction_0,TRANSFER,AED,4.55,COMPLETED,OUTBOUND,user_898,4,2018,3,1993,GB,Belfast,0,STANDARD,1,1,1,1,2018,11,Android,METAL_GAME_START,EMAIL,FAILED,2,2019,24
2,transaction_378,TRANSFER,AED,14.47,COMPLETED,OUTBOUND,user_898,4,2018,8,1993,GB,Belfast,0,STANDARD,1,1,1,1,2018,11,Android,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,11,2018,28
3,transaction_378,TRANSFER,AED,14.47,COMPLETED,OUTBOUND,user_898,4,2018,8,1993,GB,Belfast,0,STANDARD,1,1,1,1,2018,11,Android,METAL_GAME_START,EMAIL,FAILED,2,2019,24
4,transaction_379,ATM,AED,41.33,COMPLETED,OUTBOUND,user_898,4,2018,7,1993,GB,Belfast,0,STANDARD,1,1,1,1,2018,11,Android,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,11,2018,28


In [0]:
# df_main.groupby('brand')['transaction_id'].count()

In [0]:
# plt.rcParams['figure.figsize']= 15,8
# sns.countplot(df_main['transactions_currency'])
# plt.show()

In [68]:
who

Counter	 col	 device	 df1	 df2	 df3	 df_main	 go	 notification	 
np	 os	 pd	 plt	 pp	 py	 sns	 tls	 transaction	 
users	 warnings	 


In [0]:
del df1,df2,df3

In [70]:
df_main.groupby('user_id')['transaction_id'].count().reset_index()

,user_id,transaction_id
0,user_0,3112
1,user_1,903
2,user_10,6318
3,user_100,532
4,user_1000,702
...,...,...
18332,user_9995,4920
18333,user_9996,1440
18334,user_9997,1079
18335,user_9998,8


In [0]:
d = df_main.groupby('user_id')['transaction_id'].count().reset_index()

d.columns = ['user_id','tran_count_each_user']

df_main = pd.merge(df_main,d,on='user_id',how='left')

df_main.head()

del d

In [72]:
df_main.shape

(17984549, 29)

In [73]:
df_main['transaction_id'].nunique()

2707436

In [0]:
#  pp.ProfileReport(df_main)

In [0]:
# attributes_notifications_marketing_email is highly correlated with attributes_notifications_marketing_push (ρ = 0.92296)
df_main.drop('attributes_notifications_marketing_push',axis=1,inplace=True)

In [76]:
df_main.head(3)

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,direction,user_id,t_month,t_year,t_day,birth_year,country,city,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_email,num_contacts,u_month,u_year,u_day,brand,reason,channel,status,n_month,n_year,n_day,tran_count_each_user
0,transaction_0,TRANSFER,AED,4.55,COMPLETED,OUTBOUND,user_898,4,2018,3,1993,GB,Belfast,0,STANDARD,1,1,1,2018,11,Android,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,11,2018,28,522
1,transaction_0,TRANSFER,AED,4.55,COMPLETED,OUTBOUND,user_898,4,2018,3,1993,GB,Belfast,0,STANDARD,1,1,1,2018,11,Android,METAL_GAME_START,EMAIL,FAILED,2,2019,24,522
2,transaction_378,TRANSFER,AED,14.47,COMPLETED,OUTBOUND,user_898,4,2018,8,1993,GB,Belfast,0,STANDARD,1,1,1,2018,11,Android,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,11,2018,28,522


In [0]:
# Which month we have most of the Transactions
# Which month we have sent most of the notification

In [0]:
# # Monthwise user creations,user notifications ,user transactions 
# sns.set_style('dark')
# mylevels = ['u_month','n_month','t_month']
# plt.hist([df_main['u_month'],df_main['n_month'],df_main['t_month']],stacked=True,bins=12,rwidth=1,label=mylevels, color = ["skyblue",'pink','lightgreen'])
# plt.legend()
# plt.show()

In [0]:
# Per Year user creations,user notifications ,user transactions
# plt.rcParams['figure.figsize']=6,3 
# sns.set_style('dark')
# mylevels = ['u_year','n_year','t_year']
# plt.hist([df_main['u_year'],df_main['n_year'],df_main['t_year']],stacked=True,bins=2,rwidth=1,label=mylevels)
# plt.legend()
# plt.show()

# We can see we dnt have the data in 2019 which have users created

In [0]:
# # Per Day user creations,user notifications ,user transactions 
# plt.rcParams['figure.figsize']=15,8
# sns.set_style('dark')
# mylevels = ['u_day','n_day','t_day']
# plt.hist([df_main['u_day'],df_main['n_day'],df_main['t_day']],stacked=True,bins=31,rwidth=1,label=mylevels, color = ["lightpink",'orange','skyblue'])
# plt.legend()
# plt.show()

In [81]:
df_main.head()

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,direction,user_id,t_month,t_year,t_day,birth_year,country,city,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_email,num_contacts,u_month,u_year,u_day,brand,reason,channel,status,n_month,n_year,n_day,tran_count_each_user
0,transaction_0,TRANSFER,AED,4.55,COMPLETED,OUTBOUND,user_898,4,2018,3,1993,GB,Belfast,0,STANDARD,1,1,1,2018,11,Android,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,11,2018,28,522
1,transaction_0,TRANSFER,AED,4.55,COMPLETED,OUTBOUND,user_898,4,2018,3,1993,GB,Belfast,0,STANDARD,1,1,1,2018,11,Android,METAL_GAME_START,EMAIL,FAILED,2,2019,24,522
2,transaction_378,TRANSFER,AED,14.47,COMPLETED,OUTBOUND,user_898,4,2018,8,1993,GB,Belfast,0,STANDARD,1,1,1,2018,11,Android,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,11,2018,28,522
3,transaction_378,TRANSFER,AED,14.47,COMPLETED,OUTBOUND,user_898,4,2018,8,1993,GB,Belfast,0,STANDARD,1,1,1,2018,11,Android,METAL_GAME_START,EMAIL,FAILED,2,2019,24,522
4,transaction_379,ATM,AED,41.33,COMPLETED,OUTBOUND,user_898,4,2018,7,1993,GB,Belfast,0,STANDARD,1,1,1,2018,11,Android,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,11,2018,28,522


In [82]:
# Transaction made by each user each month
d = df_main.groupby(['user_id','t_month','t_year'])['transaction_id'].count().reset_index()

d.head()

,user_id,t_month,t_year,transaction_id
0,user_0,1,2018,152
1,user_0,1,2019,56
2,user_0,2,2018,112
3,user_0,2,2019,168
4,user_0,3,2018,328


In [83]:
# Transaction made by each user each month
d1 = df_main.groupby(['user_id','u_year','u_month','t_year','t_month'])['transaction_id'].count().reset_index()

d1.head(20)

,user_id,u_year,u_month,t_year,t_month,transaction_id
0,user_0,2018,1,2018,1,152
1,user_0,2018,1,2018,2,112
2,user_0,2018,1,2018,3,328
3,user_0,2018,1,2018,4,96
4,user_0,2018,1,2018,5,144
5,user_0,2018,1,2018,6,224
6,user_0,2018,1,2018,7,376
7,user_0,2018,1,2018,8,168
8,user_0,2018,1,2018,9,40
9,user_0,2018,1,2018,10,216


In [84]:
# We can see here most of the users are created in 2018 and only 22625 were created in 2019
df_main.groupby(['u_year','u_month'])['user_id'].count()

u_year  u_month
2018    1          1939590
        2          1285802
        3          2428443
        4          1418007
        5          1802333
        6          1719792
        7          1472975
        8          1922154
        9          1316676
        10          906287
        11          805208
        12          944657
2019    1            22625
Name: user_id, dtype: int64

In [0]:
# plt.rcParams['figure.figsize'] = 9,6
# sns.countplot(df_main['u_year'])
# plt.show()

In [86]:
d2 = df_main.groupby(['user_id','u_year','u_month','t_year','t_month'])['transaction_id'].count().reset_index()

d2.head(10)

,user_id,u_year,u_month,t_year,t_month,transaction_id
0,user_0,2018,1,2018,1,152
1,user_0,2018,1,2018,2,112
2,user_0,2018,1,2018,3,328
3,user_0,2018,1,2018,4,96
4,user_0,2018,1,2018,5,144
5,user_0,2018,1,2018,6,224
6,user_0,2018,1,2018,7,376
7,user_0,2018,1,2018,8,168
8,user_0,2018,1,2018,9,40
9,user_0,2018,1,2018,10,216


In [0]:
# Define a target metric to measure user engagement. How would you define an ​engaged vs. ​unengaged​ user?

# If user was created in 2018 and he has not done much transaction in 2018 and 2019 he will treated as unengaged user if so he is engaged user.

In [88]:
# These are the users which are identified as unengaged users as there numbers of trasactions are less  than 1 transactions per month
unengaged_users = d2[d2['transaction_id']<12]

unengaged_users.head()

,user_id,u_year,u_month,t_year,t_month,transaction_id
50,user_100,2018,1,2018,5,4
53,user_100,2018,1,2018,9,8
114,user_10007,2018,8,2019,3,9
130,user_10009,2018,8,2019,5,7
133,user_1001,2018,1,2018,6,4


In [0]:
# Now I will make my Target class as Engaged or Unengaged in terms of Zero and One where Zero represents Engaged and 1 represent unengaged

In [90]:
unengaged_users.isna().sum()

user_id           0
u_year            0
u_month           0
t_year            0
t_month           0
transaction_id    0
dtype: int64

In [0]:
df_main_new = df_main.copy()

In [0]:
# [(count,userid) for (count,userid) in enumerate(unengaged_users_list)]

In [0]:
# [userid for userid in unengaged_users_list if userid in list(df_main_new['user_id'])]

In [94]:
# IF the user was created in 2018 and we have data till 2019 still if he has done total trnsacation less than 12 will be considered as unengaged
df_main_new[(df_main_new['tran_count_each_user']<=12) & (df_main_new['u_year']==2018)]



,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,direction,user_id,t_month,t_year,t_day,birth_year,country,city,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_email,num_contacts,u_month,u_year,u_day,brand,reason,channel,status,n_month,n_year,n_day,tran_count_each_user
411003,transaction_2283,CARD_PAYMENT,SEK,0.75,COMPLETED,OUTBOUND,user_830,3,2018,29,1981,SE,Kungälv,0,STANDARD,2,0,1,2018,23,Apple,NO_INITIAL_CARD_ORDER,EMAIL,SENT,1,2018,27,12
411004,transaction_2283,CARD_PAYMENT,SEK,0.75,COMPLETED,OUTBOUND,user_830,3,2018,29,1981,SE,Kungälv,0,STANDARD,2,0,1,2018,23,Apple,FIFTH_PAYMENT_PROMO,EMAIL,SENT,2,2019,12,12
411005,transaction_2283,CARD_PAYMENT,SEK,0.75,COMPLETED,OUTBOUND,user_830,3,2018,29,1981,SE,Kungälv,0,STANDARD,2,0,1,2018,23,Apple,FIFTH_PAYMENT_PROMO,PUSH,SENT,2,2019,12,12
411006,transaction_2283,CARD_PAYMENT,SEK,0.75,COMPLETED,OUTBOUND,user_830,3,2018,29,1981,SE,Kungälv,0,STANDARD,2,0,1,2018,23,Apple,NO_INITIAL_CARD_ORDER,EMAIL,SENT,2,2018,7,12
411007,transaction_2283,CARD_PAYMENT,SEK,0.75,COMPLETED,OUTBOUND,user_830,3,2018,29,1981,SE,Kungälv,0,STANDARD,2,0,1,2018,23,Apple,NO_INITIAL_CARD_ORDER,EMAIL,SENT,2,2018,1,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17984544,transaction_2722402,TOPUP,EUR,0.85,FAILED,INBOUND,user_17490,5,2019,10,1985,FR,Paris,0,STANDARD,1,0,12,2018,9,Apple,NO_INITIAL_FREE_PROMOPAGE_CARD_ORDER,EMAIL,SENT,1,2019,1,3
17984545,transaction_2722402,TOPUP,EUR,0.85,FAILED,INBOUND,user_17490,5,2019,10,1985,FR,Paris,0,STANDARD,1,0,12,2018,9,Apple,NO_INITIAL_FREE_PROMOPAGE_CARD_ORDER,EMAIL,SENT,1,2019,5,3
17984546,transaction_2730860,TRANSFER,EUR,12.77,COMPLETED,INBOUND,user_15011,5,2019,7,1993,IE,Killiney,0,STANDARD,1,52,10,2018,11,Apple,NO_INITIAL_FREE_PROMOPAGE_CARD_ORDER,EMAIL,SENT,11,2018,5,3
17984547,transaction_2730860,TRANSFER,EUR,12.77,COMPLETED,INBOUND,user_15011,5,2019,7,1993,IE,Killiney,0,STANDARD,1,52,10,2018,11,Apple,NO_INITIAL_FREE_PROMOPAGE_CARD_ORDER,EMAIL,SENT,11,2018,1,3


In [0]:
# Unengaged =1
# Engaged =0
df_main_new['engagedORUnengaged'] = np.where((df_main_new['tran_count_each_user']<=12) & (df_main_new['u_year']==2018),1 ,0)

In [96]:
df_main_new['engagedORUnengaged'].value_counts()

0    17976082
1        8467
Name: engagedORUnengaged, dtype: int64

In [0]:
# sns.countplot(df_main_new['engagedORUnengaged'])

In [0]:
# Our Dataset is Highly Skewed so that needs to be treated first 
# Our Data has catagorical variables that needs to be converted to numerical

In [99]:
df_main_new.dtypes

transaction_id                               object
transactions_type                            object
transactions_currency                        object
amount_usd                                  float64
transactions_state                           object
direction                                    object
user_id                                      object
t_month                                       int64
t_year                                        int64
t_day                                         int64
birth_year                                    int64
country                                      object
city                                         object
user_settings_crypto_unlocked                 int64
plan                                         object
attributes_notifications_marketing_email      int64
num_contacts                                  int64
u_month                                       int64
u_year                                        int64
u_day       

In [100]:
df_main_new['direction'].value_counts()

OUTBOUND    14534768
INBOUND      3449781
Name: direction, dtype: int64

In [101]:
# Sepearting catagorical and numerical columns 

cat_columns = [col for col in df_main_new.select_dtypes('object') if col not in ['transaction_id','user_id','city','transactions_currency','country']]

num_columns =df_main_new.select_dtypes(['float64','int64']).columns

print(cat_columns)

print('\n')

print(num_columns)

['transactions_type', 'transactions_state', 'direction', 'plan', 'brand', 'reason', 'channel', 'status']


Index(['amount_usd', 't_month', 't_year', 't_day', 'birth_year',
       'user_settings_crypto_unlocked',
       'attributes_notifications_marketing_email', 'num_contacts', 'u_month',
       'u_year', 'u_day', 'n_month', 'n_year', 'n_day', 'tran_count_each_user',
       'engagedORUnengaged'],
      dtype='object')


In [0]:
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import auc,roc_auc_score,accuracy_score,confusion_matrix
# from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
# from sklearn.linear_model import LogisticRegression
# from imblearn.over_sampling import SMOTE

In [103]:
who

Counter	 cat_columns	 col	 d	 d1	 d2	 device	 df_main	 df_main_new	 
go	 notification	 np	 num_columns	 os	 pd	 plt	 pp	 py	 
sns	 tls	 transaction	 unengaged_users	 users	 warnings	 


In [0]:
del d,d1,d2,df_main,notification,transaction,users,device

In [0]:
#we have to drop the column transaction_id,user_id,city due to very high cardinality

# del df_main_new['transaction_id']
# del df_main_new['user_id']
# del df_main_new['city']

In [0]:
#Checking the correlation of each variables

# colormap = plt.cm.magma
# plt.figure(figsize=(16,12))
# plt.title('Pearson correlation of continuous features', y=1.05, size=15)
# sns.heatmap(df_main_new.corr(),linewidths=0.1,vmax=1.0, square=True, 
#             cmap=colormap, linecolor='white', annot=True)

In [107]:
df_main_new.groupby('transactions_type')['engagedORUnengaged'].mean()*100000

transactions_type
ATM              10.066294
CARD_PAYMENT      5.570988
CARD_REFUND       1.295874
CASHBACK          0.196858
EXCHANGE         19.563656
FEE             357.033441
REFUND           95.941667
TAX               0.000000
TOPUP           220.257282
TRANSFER         43.261572
Name: engagedORUnengaged, dtype: float64

In [108]:
df_main_new.columns

Index(['transaction_id', 'transactions_type', 'transactions_currency',
       'amount_usd', 'transactions_state', 'direction', 'user_id', 't_month',
       't_year', 't_day', 'birth_year', 'country', 'city',
       'user_settings_crypto_unlocked', 'plan',
       'attributes_notifications_marketing_email', 'num_contacts', 'u_month',
       'u_year', 'u_day', 'brand', 'reason', 'channel', 'status', 'n_month',
       'n_year', 'n_day', 'tran_count_each_user', 'engagedORUnengaged'],
      dtype='object')

In [109]:
df_main_new.head()

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,direction,user_id,t_month,t_year,t_day,birth_year,country,city,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_email,num_contacts,u_month,u_year,u_day,brand,reason,channel,status,n_month,n_year,n_day,tran_count_each_user,engagedORUnengaged
0,transaction_0,TRANSFER,AED,4.55,COMPLETED,OUTBOUND,user_898,4,2018,3,1993,GB,Belfast,0,STANDARD,1,1,1,2018,11,Android,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,11,2018,28,522,0
1,transaction_0,TRANSFER,AED,4.55,COMPLETED,OUTBOUND,user_898,4,2018,3,1993,GB,Belfast,0,STANDARD,1,1,1,2018,11,Android,METAL_GAME_START,EMAIL,FAILED,2,2019,24,522,0
2,transaction_378,TRANSFER,AED,14.47,COMPLETED,OUTBOUND,user_898,4,2018,8,1993,GB,Belfast,0,STANDARD,1,1,1,2018,11,Android,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,11,2018,28,522,0
3,transaction_378,TRANSFER,AED,14.47,COMPLETED,OUTBOUND,user_898,4,2018,8,1993,GB,Belfast,0,STANDARD,1,1,1,2018,11,Android,METAL_GAME_START,EMAIL,FAILED,2,2019,24,522,0
4,transaction_379,ATM,AED,41.33,COMPLETED,OUTBOUND,user_898,4,2018,7,1993,GB,Belfast,0,STANDARD,1,1,1,2018,11,Android,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,11,2018,28,522,0


In [110]:
df_main_new.groupby('status')['transaction_id'].count()

status
FAILED     4381330
SENT      13603219
Name: transaction_id, dtype: int64

In [0]:
# pp.ProfileReport(df_main_new)

In [0]:
# transaction_id has a high cardinality: 2707436 distinct values Warning
# amount_usd is highly skewed (γ1 = 744.11) Skewed
# amount_usd has 384656 / 2.1% zeros Zeros
# user_id has a high cardinality: 18337 distinct values Warning
# city has a high cardinality: 5744 distinct values Warning
# attributes_notifications_marketing_email has 1880025 / 10.5% zeros Zeros
# num_contacts has 2988128 / 16.6% zeros Zeros

In [0]:
# Will drop variables with high cardinality

# df_main_new.drop(['transaction_id','user_id','city','num_contacts'],axis=1,inplace=True)

In [114]:
df_main_new.head()

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,direction,user_id,t_month,t_year,t_day,birth_year,country,city,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_email,num_contacts,u_month,u_year,u_day,brand,reason,channel,status,n_month,n_year,n_day,tran_count_each_user,engagedORUnengaged
0,transaction_0,TRANSFER,AED,4.55,COMPLETED,OUTBOUND,user_898,4,2018,3,1993,GB,Belfast,0,STANDARD,1,1,1,2018,11,Android,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,11,2018,28,522,0
1,transaction_0,TRANSFER,AED,4.55,COMPLETED,OUTBOUND,user_898,4,2018,3,1993,GB,Belfast,0,STANDARD,1,1,1,2018,11,Android,METAL_GAME_START,EMAIL,FAILED,2,2019,24,522,0
2,transaction_378,TRANSFER,AED,14.47,COMPLETED,OUTBOUND,user_898,4,2018,8,1993,GB,Belfast,0,STANDARD,1,1,1,2018,11,Android,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,11,2018,28,522,0
3,transaction_378,TRANSFER,AED,14.47,COMPLETED,OUTBOUND,user_898,4,2018,8,1993,GB,Belfast,0,STANDARD,1,1,1,2018,11,Android,METAL_GAME_START,EMAIL,FAILED,2,2019,24,522,0
4,transaction_379,ATM,AED,41.33,COMPLETED,OUTBOUND,user_898,4,2018,7,1993,GB,Belfast,0,STANDARD,1,1,1,2018,11,Android,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,11,2018,28,522,0


In [0]:
from sklearn.model_selection import train_test_split
df_train ,df_test =train_test_split(df_main_new ,test_size =0.3 )

In [116]:
df_train.head()

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,direction,user_id,t_month,t_year,t_day,birth_year,country,city,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_email,num_contacts,u_month,u_year,u_day,brand,reason,channel,status,n_month,n_year,n_day,tran_count_each_user,engagedORUnengaged
9866137,transaction_1056526,TOPUP,EUR,42.55,COMPLETED,INBOUND,user_976,2,2018,22,1966,FR,Scionzier,1,STANDARD,2,1,1,2018,28,Apple,NO_INITIAL_CARD_ORDER,EMAIL,SENT,1,2018,31,1566,0
4237212,transaction_965956,TOPUP,GBP,200.00,COMPLETED,INBOUND,user_5417,12,2018,29,1995,GB,Gravesend,0,STANDARD,2,3,5,2018,20,Apple,PREMIUM_ENGAGEMENT_INACTIVE_CARD,PUSH,SENT,9,2018,29,1704,0
5987532,transaction_168962,CARD_PAYMENT,GBP,4.55,COMPLETED,OUTBOUND,user_7407,4,2019,9,1991,GB,London,0,STANDARD,2,41,6,2018,29,Apple,FIFTH_PAYMENT_PROMO,EMAIL,FAILED,2,2019,19,4044,0
3745741,transaction_1256161,CARD_PAYMENT,GBP,2.70,COMPLETED,OUTBOUND,user_4578,11,2018,8,1990,GB,London,1,STANDARD,1,29,4,2018,5,Apple,METAL_RESERVE_PLAN,SMS,SENT,10,2018,17,8195,0
4866248,transaction_1592690,CARD_PAYMENT,EUR,11.90,COMPLETED,OUTBOUND,user_4401,9,2018,6,1965,GB,Tonbridge,0,STANDARD,1,49,4,2018,12,Android,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,12,2018,31,625,0


In [0]:
# mf = mutual_info_classif(train_float.values,train.target.values,n_neighbors=3, random_state=17 )
# print(mf)

In [0]:
# a_dummies= pd.get_dummies(df_main_new["transactions_type"],drop_first=True)
# a_dummies.head()

In [0]:
# b_dummies= pd.get_dummies(data = df_main_new,columns = ["transactions_currency",'transactions_state','direction'],drop_first=True)
# b_dummies.head()

In [0]:
# df_main_new=pd.concat([df_main_new,b_dummies],1)


In [0]:
# df_main_new=df_main_new.drop(["transactions_currency",'transactions_state','direction'],1)

In [0]:
# c_dummies= pd.get_dummies(data = df_main_new,columns = ["transactions_type",'plan','brand','reason','channel','status'],drop_first=True)
# c_dummies.head()

In [0]:
# df_main_new=pd.concat([df_main_new,c_dummies],1)

In [0]:
# Creating dummies for n-1 combinations(Leaving refund and transfer)
 
# for i in range(len(df_main_new)):
#     if df_main_new["transactions_type"][i] in ["CASHBACK","TAX","CARD_REFUND","CARD_PAYMENT"]:
#         df_main_new.loc[i,"transactions_type"]="TT_CTCC"
#     if df_main_new["transactions_type"][i] in ["ATM","EXCHANGE"]:
#         df_main_new.loc[i,"transactions_type"]="TT_AE"
#     if df_main_new["transactions_type"][i]==["FEE",'TOPUP']:
#         df_main_new.loc[i,"transactions_type"]="TT_FT"
# ab_dummies=pd.get_dummies(df_main_new["transactions_type"])
# # ab_dummies.head()

In [0]:
# df_main_new=pd.concat([df_main_new,ab_dummies],1)
# df_main_new=df_main_new.drop(["transactions_type"],1)

In [0]:
# from sklearn.model_selection import train_test_split
# df_train ,df_test = train_test_split(df_main_new,test_size =0.3,random_state=2)

In [0]:
# you can use following method if you want to ignore categories with too low frequencies , 
# ignoring categories with low frequencies however will result in fewer columns without 
# affecting model performance too much .

# for col in cat_columns:
#     freqs=df_train[col].value_counts()
#     k=freqs.index[freqs>50000][:-1]
#     for cat in k:
#         name=str(col)+'_'+str(cat)
#         df_train[name]=(df_train[col]==cat).astype(int)
#     del df_train[col]
#     print(col)
    

In [126]:
who

Counter	 cat_columns	 col	 df_main_new	 df_test	 df_train	 go	 np	 num_columns	 
os	 pd	 plt	 pp	 py	 sns	 tls	 train_test_split	 unengaged_users	 
warnings	 


In [0]:
# del df_main_new	

In [0]:
# df_train.drop(['transactions_currency','country'],axis=1,inplace=True)

In [0]:
# df_test.drop(['transactions_currency','country'],axis=1,inplace=True)

In [130]:
df_train.shape,df_test.shape

((12589184, 29), (5395365, 29))

In [131]:
Counter(df_train.dtypes.values)

Counter({dtype('int64'): 15, dtype('float64'): 1, dtype('O'): 13})

In [0]:
# from sklearn.ensemble import RandomForestClassifier

In [0]:
# rf = RandomForestClassifier(random_state=0,verbose=20)
# rf.fit(df_train.drop(['engagedORUnengaged'],axis=1), df_train['engagedORUnengaged'])
# features = df_train.drop(['engagedORUnengaged'],axis=1).columns.values
# print("----- Training Done -----")

In [0]:
# colormap = plt.cm.magma
# plt.figure(figsize=(16,12))
# plt.title('Pearson correlation of continuous features', y=1.05, size=15)
# sns.heatmap(train_float.corr(),linewidths=0.1,vmax=1.0, square=True, 
#             cmap=colormap, linecolor='white', annot=True)

In [0]:
# colormap = plt.cm.magma
# plt.figure(figsize=(16,12))
# plt.title('Pearson correlation of continuous features', y=1.05, size=15)
# sns.heatmap(train_int.corr(),linewidths=0.1,vmax=1.0, square=True, 
#             cmap=colormap, linecolor='white', annot=False)

In [0]:
# #Needs to Reduce the Dimension of the data to work 

# print('\nLoading data ...')
# # X = df_train.drop(['engagedORUnengaged'], axis=1).values
# # y = df_train['engagedORUnengaged'].values.astype(np.int8)
# target_names = np.unique(df_train['engagedORUnengaged'].values.astype(np.int8))
# print('\nThere are %d unique target valuess in this dataset:' % (len(target_names)), target_names)

In [137]:
df_train.columns

Index(['transaction_id', 'transactions_type', 'transactions_currency',
       'amount_usd', 'transactions_state', 'direction', 'user_id', 't_month',
       't_year', 't_day', 'birth_year', 'country', 'city',
       'user_settings_crypto_unlocked', 'plan',
       'attributes_notifications_marketing_email', 'num_contacts', 'u_month',
       'u_year', 'u_day', 'brand', 'reason', 'channel', 'status', 'n_month',
       'n_year', 'n_day', 'tran_count_each_user', 'engagedORUnengaged'],
      dtype='object')

In [0]:
# from sklearn.decomposition import PCA

In [0]:
# n_comp = 10
# # PCA
# print('\nRunning PCA ...')
# pca = PCA(n_components=n_comp)
# X_pca = pca.fit_transform(df_train.drop(['engagedORUnengaged'], axis=1).values)
# print('Explained variance: %.4f' % pca.explained_variance_ratio_.sum())

In [0]:
# print('Individual variance contributions:')
# for j in range(n_comp):
#     print(pca.explained_variance_ratio_[j])

In [0]:
# mf = mutual_info_classif(train_float.values,train.target.values,n_neighbors=3, random_state=17 )
# print(mf)

In [0]:
# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier(n_estimators=150, max_depth=8, min_samples_leaf=4, max_features=0.2, n_jobs=-1, random_state=0,verbose=20)
# rf.fit(df_main_new.drop(['engagedORUnengaged'],axis=1), df_main_new['engagedORUnengaged'])
# features = df_main_new.drop(['engagedORUnengaged'],axis=1).columns.values
# print("----- Training Done -----")

In [0]:
# x_train ,x_test ,y_train,y_test = train_test_split(df_main_new , test_size = 0.3,random_state = 2 )

# list(zip(df_train.shape , df_test.shape))

In [0]:
# x_train , y_train =df_train.drop('engagedORUnengaged',axis=1) ,df_train['engagedORUnengaged']

In [0]:
# x_test ,y_test = df_test.drop('engagedORUnengaged',axis =1) ,df_test['engagedORUnengaged']

In [0]:
# y_train.value_counts()

In [0]:
# smt = SMOTE()
# x_train, y_train = smt.fit_sample(x_train, y_train)

In [0]:
# np.bincount(y_train)

## **`Feature Engineering`**

In [149]:
df_train.reset_index(drop=True)

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,direction,user_id,t_month,t_year,t_day,birth_year,country,city,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_email,num_contacts,u_month,u_year,u_day,brand,reason,channel,status,n_month,n_year,n_day,tran_count_each_user,engagedORUnengaged
0,transaction_1056526,TOPUP,EUR,42.55,COMPLETED,INBOUND,user_976,2,2018,22,1966,FR,Scionzier,1,STANDARD,2,1,1,2018,28,Apple,NO_INITIAL_CARD_ORDER,EMAIL,SENT,1,2018,31,1566,0
1,transaction_965956,TOPUP,GBP,200.00,COMPLETED,INBOUND,user_5417,12,2018,29,1995,GB,Gravesend,0,STANDARD,2,3,5,2018,20,Apple,PREMIUM_ENGAGEMENT_INACTIVE_CARD,PUSH,SENT,9,2018,29,1704,0
2,transaction_168962,CARD_PAYMENT,GBP,4.55,COMPLETED,OUTBOUND,user_7407,4,2019,9,1991,GB,London,0,STANDARD,2,41,6,2018,29,Apple,FIFTH_PAYMENT_PROMO,EMAIL,FAILED,2,2019,19,4044,0
3,transaction_1256161,CARD_PAYMENT,GBP,2.70,COMPLETED,OUTBOUND,user_4578,11,2018,8,1990,GB,London,1,STANDARD,1,29,4,2018,5,Apple,METAL_RESERVE_PLAN,SMS,SENT,10,2018,17,8195,0
4,transaction_1592690,CARD_PAYMENT,EUR,11.90,COMPLETED,OUTBOUND,user_4401,9,2018,6,1965,GB,Tonbridge,0,STANDARD,1,49,4,2018,12,Android,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,12,2018,31,625,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12589179,transaction_2662627,ATM,EUR,42.55,COMPLETED,OUTBOUND,user_10786,2,2019,24,1988,FR,Antibes,1,PREMIUM,1,3,8,2018,27,Android,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,10,2018,6,4202,0
12589180,transaction_1665865,CARD_PAYMENT,EUR,8.94,COMPLETED,OUTBOUND,user_9701,1,2019,26,1988,MT,Attard,1,STANDARD,1,109,7,2018,12,Android,ENGAGEMENT_SPLIT_BILL_RESTAURANT,EMAIL,SENT,12,2018,28,1884,0
12589181,transaction_2615714,CARD_PAYMENT,EUR,4.63,COMPLETED,OUTBOUND,user_11439,12,2018,1,1991,LT,Vilnius,1,STANDARD,1,58,8,2018,22,Apple,METAL_RESERVE_PLAN,SMS,SENT,10,2018,14,6750,0
12589182,transaction_921383,CARD_PAYMENT,GBP,42.25,COMPLETED,OUTBOUND,user_9842,11,2018,19,1956,GB,Larne,0,PREMIUM,0,0,8,2018,18,Apple,NO_INITIAL_CARD_ORDER,EMAIL,FAILED,9,2018,8,4551,0


In [150]:
target =  df_train[['user_id','engagedORUnengaged']]

len(target)

12589184

In [151]:
c=['count','nunique']
n=['mean','max','min','sum','std']
agg_c={'transaction_id':c,'transactions_type':c,'transactions_currency':c,'transactions_state':c,'amount_usd':n,'transactions_state':c,\
       'direction':c,'t_month':c,'t_year':c,'t_day':c,\
       'birth_year':c,'country':c,'city':c,'user_settings_crypto_unlocked':c,'plan':c,'attributes_notifications_marketing_email':c,\
       'num_contacts':n,'u_month':c,'u_year':c,'u_day':c,'brand':c,'reason':c,'channel':c,\
       'status':c,'n_month':c,'n_year':c,'n_day':c ,'tran_count_each_user':n}
df_train=df_train.groupby(['user_id']).agg(agg_c)
df_train.head()

transaction_id         transactions_type  ... tran_count_each_user               
                   count nunique             count  ...                  min       sum  std
user_id                                             ...                                    
user_0              2193     389              2193  ...                 3112   6824616  0.0
user_1               642     129               642  ...                  903    579726  0.0
user_10             4400    1051              4400  ...                 6318  27799200  0.0
user_100             369     131               369  ...                  532    196308  0.0
user_1000            500      39               500  ...                  702    351000  0.0

[5 rows x 63 columns]

In [152]:
df_train.columns=['R_' + '_'.join(col).strip() for col in df_train.columns.values]
df_train.reset_index(inplace=True)
df_train.head()

,user_id,R_transaction_id_count,R_transaction_id_nunique,R_transactions_type_count,R_transactions_type_nunique,R_transactions_currency_count,R_transactions_currency_nunique,R_transactions_state_count,R_transactions_state_nunique,R_amount_usd_mean,R_amount_usd_max,R_amount_usd_min,R_amount_usd_sum,R_amount_usd_std,R_direction_count,R_direction_nunique,R_t_month_count,R_t_month_nunique,R_t_year_count,R_t_year_nunique,R_t_day_count,R_t_day_nunique,R_birth_year_count,R_birth_year_nunique,R_country_count,R_country_nunique,R_city_count,R_city_nunique,R_user_settings_crypto_unlocked_count,R_user_settings_crypto_unlocked_nunique,R_plan_count,R_plan_nunique,R_attributes_notifications_marketing_email_count,R_attributes_notifications_marketing_email_nunique,R_num_contacts_mean,R_num_contacts_max,R_num_contacts_min,R_num_contacts_sum,R_num_contacts_std,R_u_month_count,R_u_month_nunique,R_u_year_count,R_u_year_nunique,R_u_day_count,R_u_day_nunique,R_brand_count,R_brand_nunique,R_reason_count,R_reason_nunique,R_channel_count,R_channel_nunique,R_status_count,R_status_nunique,R_n_month_count,R_n_month_nunique,R_n_year_count,R_n_year_nunique,R_n_day_count,R_n_day_nunique,R_tran_count_each_user_mean,R_tran_count_each_user_max,R_tran_count_each_user_min,R_tran_count_each_user_sum,R_tran_count_each_user_std
0,user_0,2193,389,2193,7,2193,5,2193,4,34.413552,582.18,0.00,75468.92,65.765313,2193,2,2193,12,2193,2,2193,31,2193,1,2193,1,2193,1,2193,1,2193,1,2193,1,3,3,3,6579,0.0,2193,1,2193,1,2193,1,2193,1,2193,5,2193,2,2193,1,2193,5,2193,2,2193,7,3112,3112,3112,6824616,0.0
1,user_1,642,129,642,5,642,1,642,4,48.666854,350.00,0.00,31244.12,66.033638,642,2,642,11,642,2,642,24,642,1,642,1,642,1,642,1,642,1,642,1,21,21,21,13482,0.0,642,1,642,1,642,1,642,1,642,5,642,2,642,2,642,4,642,2,642,6,903,903,903,579726,0.0
2,user_10,4400,1051,4400,7,4400,1,4400,5,49.713348,2947.24,0.00,218738.73,174.978189,4400,2,4400,12,4400,2,4400,31,4400,1,4400,1,4400,1,4400,1,4400,1,4400,1,38,38,38,167200,0.0,4400,1,4400,1,4400,1,4400,1,4400,4,4400,2,4400,1,4400,4,4400,2,4400,5,6318,6318,6318,27799200,0.0
3,user_100,369,131,369,5,369,2,369,3,148.446287,709.21,0.55,54776.68,170.736592,369,2,369,9,369,2,369,29,369,1,369,1,369,1,369,1,369,1,369,1,0,0,0,0,0.0,369,1,369,1,369,1,369,1,369,3,369,2,369,1,369,3,369,1,369,3,532,532,532,196308,0.0
4,user_1000,500,39,500,4,500,3,500,4,551.876800,1000.03,1.00,275938.40,341.397577,500,2,500,4,500,1,500,14,500,1,500,1,500,1,500,1,500,1,500,1,0,0,0,0,0.0,500,1,500,1,500,1,500,1,500,4,500,3,500,2,500,8,500,2,500,14,702,702,702,351000,0.0


In [153]:
df_train.sample(10)

,user_id,R_transaction_id_count,R_transaction_id_nunique,R_transactions_type_count,R_transactions_type_nunique,R_transactions_currency_count,R_transactions_currency_nunique,R_transactions_state_count,R_transactions_state_nunique,R_amount_usd_mean,R_amount_usd_max,R_amount_usd_min,R_amount_usd_sum,R_amount_usd_std,R_direction_count,R_direction_nunique,R_t_month_count,R_t_month_nunique,R_t_year_count,R_t_year_nunique,R_t_day_count,R_t_day_nunique,R_birth_year_count,R_birth_year_nunique,R_country_count,R_country_nunique,R_city_count,R_city_nunique,R_user_settings_crypto_unlocked_count,R_user_settings_crypto_unlocked_nunique,R_plan_count,R_plan_nunique,R_attributes_notifications_marketing_email_count,R_attributes_notifications_marketing_email_nunique,R_num_contacts_mean,R_num_contacts_max,R_num_contacts_min,R_num_contacts_sum,R_num_contacts_std,R_u_month_count,R_u_month_nunique,R_u_year_count,R_u_year_nunique,R_u_day_count,R_u_day_nunique,R_brand_count,R_brand_nunique,R_reason_count,R_reason_nunique,R_channel_count,R_channel_nunique,R_status_count,R_status_nunique,R_n_month_count,R_n_month_nunique,R_n_year_count,R_n_year_nunique,R_n_day_count,R_n_day_nunique,R_tran_count_each_user_mean,R_tran_count_each_user_max,R_tran_count_each_user_min,R_tran_count_each_user_sum,R_tran_count_each_user_std
10408,user_257,21,3,21,2,21,1,21,2,4.904762,10.00,1.0,103.00,3.726993,21,2,21,1,21,1,21,1,21,1,21,1,21,1,21,1,21,1,21,1,0,0,0,0,0.0,21,1,21,1,21,1,21,1,21,4,21,2,21,2,21,3,21,2,21,7,33,33,33,693,0.0
12892,user_4875,135,21,135,4,135,2,135,4,68.837407,200.00,1.0,9293.05,71.981512,135,2,135,2,135,1,135,6,135,1,135,1,135,1,135,1,135,1,135,1,0,0,0,0,0.0,135,1,135,1,135,1,135,1,135,4,135,3,135,2,135,5,135,2,135,6,189,189,189,25515,0.0
1145,user_11083,14,6,14,4,14,1,14,3,2.388571,3.98,0.2,33.44,1.455371,14,2,14,2,14,1,14,3,14,1,14,1,14,1,14,1,14,1,14,1,6,6,6,84,0.0,14,1,14,1,14,1,14,1,14,2,14,2,14,2,14,2,14,1,14,2,18,18,18,252,0.0
4756,user_14613,726,252,726,5,726,2,726,3,25.817149,306.38,0.0,18743.25,39.393834,726,2,726,8,726,2,726,31,726,1,726,1,726,1,726,1,726,1,726,1,199,199,199,144474,0.0,726,1,726,1,726,1,726,1,726,3,726,2,726,1,726,3,726,2,726,3,1020,1020,1020,740520,0.0
1741,user_11657,191,115,191,4,191,1,191,4,14.039110,200.00,0.8,2681.47,33.615673,191,2,191,8,191,2,191,31,191,1,191,1,191,1,191,1,191,1,191,1,200,200,200,38200,0.0,191,1,191,1,191,1,191,1,191,1,191,1,191,1,191,2,191,2,191,2,246,246,246,46986,0.0
1861,user_11772,42,14,42,3,42,1,42,3,22.063810,170.21,0.0,926.68,52.573217,42,2,42,4,42,2,42,7,42,1,42,1,42,1,42,1,42,1,42,1,0,0,0,0,0.0,42,1,42,1,42,1,42,1,42,2,42,2,42,2,42,2,42,1,42,2,56,56,56,2352,0.0
3623,user_13498,989,238,989,7,989,3,989,5,125.631072,1492.83,0.0,124249.13,198.824843,989,2,989,8,989,2,989,31,989,1,989,1,989,1,989,1,989,1,989,1,0,0,0,0,0.0,989,1,989,1,989,1,989,1,989,3,989,2,989,1,989,4,989,2,989,5,1428,1428,1428,1412292,0.0
10104,user_2289,4729,1114,4729,6,4729,2,4729,5,24.669734,553.19,0.0,116663.17,68.352582,4729,2,4729,12,4729,2,4729,31,4729,1,4729,1,4729,1,4729,1,4729,1,4729,1,39,39,39,184431,0.0,4729,1,4729,1,4729,1,4729,1,4729,4,4729,2,4729,2,4729,4,4729,2,4729,4,6696,6696,6696,31665384,0.0
8113,user_17870,37,9,37,2,37,1,37,4,8.575946,74.64,0.0,317.31,23.319846,37,2,37,3,37,1,37,3,37,1,37,1,37,1,37,1,37,1,37,1,0,0,0,0,0.0,37,1,37,1,37,1,37,1,37,4,37,2,37,2,37,3,37,2,37,5,60,60,60,2220,0.0
3853,user_13716,70,8,70,3,70,1,70,3,8.996286,19.69,0.0,629.74,9.106960,70,2,70,4,70,2,70,5,70,1,70,1,70,1,70,1,70,1,70,1,32,32,32,2240,0.0,70,1,70,1,70,1,70,1,70,6,70,2,70,2,70,3,70,2,70,9,96,96,96,6720,0.0


In [0]:
df =pd.merge(df_train,target,on='user_id',how='left')

In [157]:
df.head()

,user_id,R_transaction_id_count,R_transaction_id_nunique,R_transactions_type_count,R_transactions_type_nunique,R_transactions_currency_count,R_transactions_currency_nunique,R_transactions_state_count,R_transactions_state_nunique,R_amount_usd_mean,R_amount_usd_max,R_amount_usd_min,R_amount_usd_sum,R_amount_usd_std,R_direction_count,R_direction_nunique,R_t_month_count,R_t_month_nunique,R_t_year_count,R_t_year_nunique,R_t_day_count,R_t_day_nunique,R_birth_year_count,R_birth_year_nunique,R_country_count,R_country_nunique,R_city_count,R_city_nunique,R_user_settings_crypto_unlocked_count,R_user_settings_crypto_unlocked_nunique,R_plan_count,R_plan_nunique,R_attributes_notifications_marketing_email_count,R_attributes_notifications_marketing_email_nunique,R_num_contacts_mean,R_num_contacts_max,R_num_contacts_min,R_num_contacts_sum,R_num_contacts_std,R_u_month_count,R_u_month_nunique,R_u_year_count,R_u_year_nunique,R_u_day_count,R_u_day_nunique,R_brand_count,R_brand_nunique,R_reason_count,R_reason_nunique,R_channel_count,R_channel_nunique,R_status_count,R_status_nunique,R_n_month_count,R_n_month_nunique,R_n_year_count,R_n_year_nunique,R_n_day_count,R_n_day_nunique,R_tran_count_each_user_mean,R_tran_count_each_user_max,R_tran_count_each_user_min,R_tran_count_each_user_sum,R_tran_count_each_user_std,engagedORUnengaged
0,user_0,2193,389,2193,7,2193,5,2193,4,34.413552,582.18,0.0,75468.92,65.765313,2193,2,2193,12,2193,2,2193,31,2193,1,2193,1,2193,1,2193,1,2193,1,2193,1,3,3,3,6579,0.0,2193,1,2193,1,2193,1,2193,1,2193,5,2193,2,2193,1,2193,5,2193,2,2193,7,3112,3112,3112,6824616,0.0,0
1,user_0,2193,389,2193,7,2193,5,2193,4,34.413552,582.18,0.0,75468.92,65.765313,2193,2,2193,12,2193,2,2193,31,2193,1,2193,1,2193,1,2193,1,2193,1,2193,1,3,3,3,6579,0.0,2193,1,2193,1,2193,1,2193,1,2193,5,2193,2,2193,1,2193,5,2193,2,2193,7,3112,3112,3112,6824616,0.0,0
2,user_0,2193,389,2193,7,2193,5,2193,4,34.413552,582.18,0.0,75468.92,65.765313,2193,2,2193,12,2193,2,2193,31,2193,1,2193,1,2193,1,2193,1,2193,1,2193,1,3,3,3,6579,0.0,2193,1,2193,1,2193,1,2193,1,2193,5,2193,2,2193,1,2193,5,2193,2,2193,7,3112,3112,3112,6824616,0.0,0
3,user_0,2193,389,2193,7,2193,5,2193,4,34.413552,582.18,0.0,75468.92,65.765313,2193,2,2193,12,2193,2,2193,31,2193,1,2193,1,2193,1,2193,1,2193,1,2193,1,3,3,3,6579,0.0,2193,1,2193,1,2193,1,2193,1,2193,5,2193,2,2193,1,2193,5,2193,2,2193,7,3112,3112,3112,6824616,0.0,0
4,user_0,2193,389,2193,7,2193,5,2193,4,34.413552,582.18,0.0,75468.92,65.765313,2193,2,2193,12,2193,2,2193,31,2193,1,2193,1,2193,1,2193,1,2193,1,2193,1,3,3,3,6579,0.0,2193,1,2193,1,2193,1,2193,1,2193,5,2193,2,2193,1,2193,5,2193,2,2193,7,3112,3112,3112,6824616,0.0,0


In [0]:
df.isna().sum()

In [0]:
# df_train_with_target.isna().sum()